# knn

data 

In [1]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd
import random
random.seed(2021)

In [2]:
path = "../data//train_set"
image_dir = path+"images/"
pt_dir = path+"points/"
label_path = path+"label.csv"

In [3]:
def read_data(path):
  
  # read labels
  labels = pd.read_csv(path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)

In [4]:
read_time_start=time.time()
X, Y = read_data(path)
print("Read the original dataset takes %s seconds" % round((time.time() - read_time_start),3))
X.shape, Y.shape 

Read the original dataset takes 107.318 seconds


((3000, 3003), (3000,))

In [5]:
print('majority class: %d' % np.sum(Y == 0))
print('minority class: %d' % np.sum(Y == 1))
#imbalanced dataset

majority class: 2402
minority class: 598


# oversample then split data

In [6]:
def data_preprocessing(X, Y, path):

  distances = X
  y = Y

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [7]:
X_balanced, Y_balanced = data_preprocessing(X, Y, path)
X_balanced.shape, Y_balanced.shape

((4804, 3003), (4804,))

In [8]:
print('majority class: %d' % np.sum(Y_balanced == 0))
print('minority class: %d' % np.sum(Y_balanced == 1))
#balanced dataset

majority class: 2402
minority class: 2402


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd

In [10]:
from sklearn.model_selection import train_test_split
X_balanced_train,X_balanced_test,y_balanced_train,y_balanced_test= train_test_split(X_balanced, Y_balanced,test_size=0.2,random_state=0)
X_balanced_train.shape,X_balanced_test.shape,y_balanced_train.shape,y_balanced_test.shape

((3843, 3003), (961, 3003), (3843,), (961,))

In [11]:
print('majority train class: %d' % np.sum(y_balanced_train == 0))
print('minority train class: %d' % np.sum(y_balanced_train == 1))
print('majority test class: %d' % np.sum(y_balanced_test == 0))
print('minority test class: %d' % np.sum(y_balanced_test == 1))

majority train class: 1918
minority train class: 1925
majority test class: 484
minority test class: 477


# split then oversample

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.20,random_state=0)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
print('majority train class: %d' % np.sum(Y_train == 0))
print('minority train class: %d' % np.sum(Y_train == 1))
print('majority test class: %d' % np.sum(Y_test == 0))
print('minority test class: %d' % np.sum(Y_test == 1))
#imbalanced dataset

majority train class: 1923
minority train class: 477
majority test class: 479
minority test class: 121


In [13]:
def data_preprocessing(X_train, Y_train, path):

  distances = X_train
  y = Y_train

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [14]:
X_train_balanced, y_train_balanced = data_preprocessing(X_train, Y_train, path)
X_train_balanced.shape, y_train_balanced.shape

((3846, 3003), (3846,))

In [15]:
def data_preprocessing(X_test, Y_test, path):

  distances = X_test
  y = Y_test

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [16]:
X_test_balanced, y_test_balanced = data_preprocessing(X_test, Y_test, path)
X_test_balanced.shape, y_test_balanced.shape
print('majority train class: %d' % np.sum(y_train_balanced == 0))
print('minority train class: %d' % np.sum(y_train_balanced == 1))
print('majority test class: %d' % np.sum(y_test_balanced == 0))
print('minority test class: %d' % np.sum(y_test_balanced == 1))

majority train class: 1923
minority train class: 1923
majority test class: 479
minority test class: 479


# divide into 2 chunk 3 groups 

        
    • balanced train
    
        1.(train with balanced,test with balanced)
           a.X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
           b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
        
        2.(train with balanced,test with imbalanced) 
            X_train_balanced.y_train_balanced;X_test,Y_test
      

    • imbalanced train
    
        3. (train with imbalanced,test with imbalanced)
            X_train,Y_train; X_test,Y_test 

model

In [17]:
from sklearn.neighbors import NearestCentroid
import numpy as np
from sklearn.metrics import classification_report
import pickle

In [18]:
# 1.a.X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
clf = NearestCentroid()
start_time=time.time()
clf.fit(X_balanced_train, y_balanced_train)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 0.085 seconds
Predicting test data takes 0.016 seconds
              precision    recall  f1-score   support

           0       0.56      0.54      0.55       484
           1       0.55      0.57      0.56       477

    accuracy                           0.55       961
   macro avg       0.55      0.55      0.55       961
weighted avg       0.55      0.55      0.55       961



In [22]:
#Save trained KNN model
pickle.dump(clf, open("../output/clf1a.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/clf1a.p",'rb'))

NearestCentroid()

In [20]:
# 1.b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
clf = NearestCentroid()
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 0.085 seconds
Predicting test data takes 0.016 seconds
              precision    recall  f1-score   support

           0       0.50      0.50      0.50       479
           1       0.50      0.50      0.50       479

    accuracy                           0.50       958
   macro avg       0.50      0.50      0.50       958
weighted avg       0.50      0.50      0.50       958



In [23]:
#Save trained KNN model
pickle.dump(clf, open("../output/clf1b.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/clf1b.p",'rb'))

NearestCentroid()

In [24]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
clf = NearestCentroid()
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.082 seconds
Predicting test data takes 0.0 seconds
              precision    recall  f1-score   support

           0       0.82      0.50      0.62       479
           1       0.22      0.57      0.32       121

    accuracy                           0.52       600
   macro avg       0.52      0.54      0.47       600
weighted avg       0.70      0.52      0.56       600



In [25]:
#Save trained KNN model
pickle.dump(clf, open("../output/clf2.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/clf2.p",'rb'))

NearestCentroid()

In [26]:
#3 X_train,Y_train; X_test,Y_test

clf = NearestCentroid()
start_time=time.time()
clf.fit(X_train, Y_train)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.053 seconds
Predicting test data takes 0.018 seconds
              precision    recall  f1-score   support

           0       0.83      0.53      0.65       479
           1       0.24      0.59      0.34       121

    accuracy                           0.54       600
   macro avg       0.54      0.56      0.49       600
weighted avg       0.71      0.54      0.58       600



In [27]:
#Save trained KNN model
pickle.dump(clf, open("../output/clf3.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/clf3.p",'rb'))

NearestCentroid()

In [28]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_balanced_train, y_balanced_train)
    pre=nca_pipe.predict(X_balanced_test)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_balanced_test, y_balanced_test)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(y_balanced_test, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_balanced_train, y_balanced_train)
# Pipeline(...)
pre=nca_pipe.predict(X_balanced_test)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_balanced_test, y_balanced_test)))
print('the recall of a model with k = %d is %f' % (k, recall_score(y_balanced_test, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_balanced_test)
print(classification_report(y_balanced_test,pre))

the score of a model with k = 3 is 0.688866
the recall of a model with k = 3 is 0.937107
the score of a model with k = 4 is 0.728408
the recall of a model with k = 4 is 0.899371
the score of a model with k = 5 is 0.674298
the recall of a model with k = 5 is 0.955975
the score of a model with k = 6 is 0.706556
the recall of a model with k = 6 is 0.926625
the score of a model with k = 7 is 0.672216
the recall of a model with k = 7 is 0.955975
the score of a model with k = 8 is 0.693028
the recall of a model with k = 8 is 0.941300
the score of a model with k = 9 is 0.654527
the recall of a model with k = 9 is 0.962264
the score of a model with k = 3 is 0.688866
the recall of a model with k = 3 is 0.937107
              precision    recall  f1-score   support

           0       0.88      0.44      0.59       484
           1       0.62      0.94      0.75       477

    accuracy                           0.69       961
   macro avg       0.75      0.69      0.67       961
weighted avg    

In [29]:
#Save trained KNN model
pickle.dump(nca_pipe, open("../output/nca_pipe1a.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/nca_pipe1a.p",'rb'))

Pipeline(steps=[('nca', NeighborhoodComponentsAnalysis(random_state=42)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])

In [30]:
# 1.b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train_balanced, y_train_balanced)
    pre=nca_pipe.predict(X_test_balanced)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test_balanced, y_test_balanced)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(y_test_balanced, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train_balanced, y_train_balanced)
# Pipeline(...)
pre=nca_pipe.predict(X_test_balanced)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test_balanced, y_test_balanced)))
print('the recall of a model with k = %d is %f' % (k, recall_score(y_test_balanced, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_test_balanced)
print(classification_report(y_test_balanced,pre))

the score of a model with k = 3 is 0.638831
the recall of a model with k = 3 is 0.860125
the score of a model with k = 4 is 0.655532
the recall of a model with k = 4 is 0.791232
the score of a model with k = 5 is 0.620042
the recall of a model with k = 5 is 0.891441
the score of a model with k = 6 is 0.652401
the recall of a model with k = 6 is 0.849687
the score of a model with k = 7 is 0.629436
the recall of a model with k = 7 is 0.924843
the score of a model with k = 8 is 0.646138
the recall of a model with k = 8 is 0.885177
the score of a model with k = 9 is 0.608559
the recall of a model with k = 9 is 0.931106
the score of a model with k = 3 is 0.638831
the recall of a model with k = 3 is 0.860125
              precision    recall  f1-score   support

           0       0.75      0.42      0.54       479
           1       0.60      0.86      0.70       479

    accuracy                           0.64       958
   macro avg       0.67      0.64      0.62       958
weighted avg    

In [31]:
#Save trained KNN model
pickle.dump(nca_pipe, open("../output/nca_pipe1b.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/nca_pipe1b.p",'rb'))

Pipeline(steps=[('nca', NeighborhoodComponentsAnalysis(random_state=42)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])

In [32]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train_balanced, y_train_balanced)
    pre=nca_pipe.predict(X_test)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train_balanced, y_train_balanced)
# Pipeline(...)
pre=nca_pipe.predict(X_test)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_test)
print(classification_report(Y_test,pre))

the score of a model with k = 3 is 0.481667
the recall of a model with k = 3 is 0.735537
the score of a model with k = 4 is 0.536667
the recall of a model with k = 4 is 0.603306
the score of a model with k = 5 is 0.431667
the recall of a model with k = 5 is 0.760331
the score of a model with k = 6 is 0.506667
the recall of a model with k = 6 is 0.710744
the score of a model with k = 7 is 0.438333
the recall of a model with k = 7 is 0.851240
the score of a model with k = 8 is 0.481667
the recall of a model with k = 8 is 0.776860
the score of a model with k = 9 is 0.400000
the recall of a model with k = 9 is 0.851240
the score of a model with k = 3 is 0.481667
the recall of a model with k = 3 is 0.735537
              precision    recall  f1-score   support

           0       0.86      0.42      0.56       479
           1       0.24      0.74      0.36       121

    accuracy                           0.48       600
   macro avg       0.55      0.58      0.46       600
weighted avg    

In [33]:
#Save trained KNN model
pickle.dump(nca_pipe, open("../output/nca_pipe2.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/nca_pipe2.p",'rb'))

Pipeline(steps=[('nca', NeighborhoodComponentsAnalysis(random_state=42)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])

In [34]:
#3 X_train,Y_train; X_test,Y_test
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train, Y_train)
    pre=nca_pipe.predict(X_test)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train, Y_train)
# Pipeline(...)
pre=nca_pipe.predict(X_test)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_test)
print(classification_report(Y_test,pre))

the score of a model with k = 3 is 0.776667
the recall of a model with k = 3 is 0.223140
the score of a model with k = 4 is 0.800000
the recall of a model with k = 4 is 0.090909
the score of a model with k = 5 is 0.786667
the recall of a model with k = 5 is 0.148760
the score of a model with k = 6 is 0.800000
the recall of a model with k = 6 is 0.082645
the score of a model with k = 7 is 0.790000
the recall of a model with k = 7 is 0.115702
the score of a model with k = 8 is 0.810000
the recall of a model with k = 8 is 0.082645
the score of a model with k = 9 is 0.803333
the recall of a model with k = 9 is 0.132231
the score of a model with k = 3 is 0.776667
the recall of a model with k = 3 is 0.223140
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       479
           1       0.40      0.22      0.29       121

    accuracy                           0.78       600
   macro avg       0.61      0.57      0.58       600
weighted avg    

In [35]:
#Save trained KNN model
pickle.dump(nca_pipe, open("../output/nca_pipe3.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/nca_pipe3.p",'rb'))

Pipeline(steps=[('nca', NeighborhoodComponentsAnalysis(random_state=42)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])